In [11]:
!cp -r ../../../../../vila/data/test_imgs ./data/task_images/

In [18]:
!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 68.2 MB/s eta 0:00:00:00:0100:01


## process video

In [8]:
!mkdir -p ./data/task_imgs_25/train_imgs/positive
!mkdir -p ./data/task_imgs_25/train_imgs/negative
!mkdir -p ./data/task_imgs_25/test_imgs/positive
!mkdir -p ./data/task_imgs_25/test_imgs/negative


In [20]:
!python process_video.py --input_video_folder ../../../../../vila/data/train/positive \
--output_image_folder ./data/task_imgs_25/train_imgs/positive \
--num_frm 25

Directory './data/task_imgs_25/train_imgs/positive' already exists
100%|█████████████████████████████████████████| 800/800 [12:42<00:00,  1.05it/s]


In [ ]:
!python process_video.py --input_video_folder ../../../../../vila/data/train/negative \
--output_image_folder ./data/task_imgs_25/train_imgs/negative \
--num_frm 25

Directory './data/task_imgs_25/train_imgs/negative' already exists
 46%|███████████████████                      | 372/800 [09:46<08:17,  1.16s/it]

In [ ]:
!python process_video.py --input_video_folder ../../../../../vila/data/test/positive \
--output_image_folder ./data/task_imgs_25/test_imgs/positive \
--num_frm 25

In [ ]:
!python process_video.py --input_video_folder ../../../../../vila/data/test/negative \
--output_image_folder ./data/task_imgs_25/test_imgs/negative \
--num_frm 25

## process train.json/test.json

In [3]:
## rewrite to 2.0
!python process_data.py

Data written to json
Data written to json


## train
prepare train-env: https://github.com/InternLM/InternLM-XComposer/blob/main/InternLM-XComposer-2.0/docs/install.md

In [ ]:
'''
conda create -n intern_clean python=3.9 -y
source activate intern_clean
pip3 install torch torchvision torchaudio
pip install transformers==4.33.2 timm==0.4.12 sentencepiece==0.1.99 gradio==4.13.0 markdown2==2.4.10 xlsxwriter==3.1.2 einops
# install deepspeed
pip install deepspeed

# install peft
pip install peft
'''

In [ ]:
!nohup sh finetune.sh --> trainmodelc0201.log 2>&1 &

## test

In [ ]:
!python inference_test.py

## acc calc

In [48]:
total_len = len(lines)
label_total_yes = 0
pred_total_yes = 0
total_right = 0
num = 0

for i in lines:
    pred, label = i.split(' ')[1],i.split(' ')[2]
    if pred==label:
        total_right += 1
    if pred=='Yes':
        pred_total_yes +=1
    if label=='Yes':
        label_total_yes +=1
    if label=='Yes' and pred=='Yes':
        num +=1
        
print ("acc", total_right/total_len)
print ("recall: ",  num/label_total_yes)
print ("precision: ",  num/pred_total_yes)

acc 0.81
recall:  0.835
precision:  0.7952380952380952


In [25]:
print ("acc", total_right/total_len)

acc 0.475


In [26]:
print ("recall: ",  num/label_total_yes)

ZeroDivisionError: division by zero

In [28]:
print ("precision: ",  num/pred_total_yes)

precision:  0.0


In [2]:
!pip install transformers 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 79.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 117.8 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
import json
from tqdm import tqdm

torch.set_grad_enabled(False)

def load_model(model_nameorpath):
    # init model and tokenizer
    model = AutoModel.from_pretrained(model_nameorpath, trust_remote_code=True).cuda().eval()
    tokenizer = AutoTokenizer.from_pretrained(model_nameorpath, trust_remote_code=True)
    return model, tokenizer

def single_infer(img_path,model,tokenizer):
    text = '<ImageHere>仔细描述这张图'
    image = img_path
    with torch.cuda.amp.autocast():
      response, _ = model.chat(tokenizer, query=text, image=image, history=[], do_sample=False)

    return response


ModuleNotFoundError: No module named 'transformers'

In [ ]:
model_nameorpath = ''
img_path = ''
model, tokenizer = load_model(model_nameorpath)
res = single_infer(img_path,model,tokenizer)